In [ ]:
# !pip install mlem torchvision tensorflow numpy torch torchaudio --upgrade
# !pip install mlem==0.4.6 --no-deps
# !pip install iterative-telemetry==0.0.7 --ignore-requires-python --no-deps
# !pip install pydantic==1.10.2 --no-deps

In [ ]:
from __future__ import print_function
from __future__ import division

import os
import copy
import typing
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from tqdm.notebook import tqdm
from collections import OrderedDict

# Нейронки
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, transforms
from torchvision import transforms as torch_transforms
from torchvision import io as torch_io
from torchvision import models as torch_models

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

# Параметры

In [ ]:
class WorkingMode:
    TRAIN: str = "train"
    VAL: str = "val"

In [ ]:
INPUT_DIR: Path = Path("/kaggle/input/")
OUTPUT_DIR: Path = Path("/kaggle/working/")

# FINAL_DATASET_DIR: Path = INPUT_DIR / "art-price" / "dataset"
FINAL_DATASET_DIR: Path = INPUT_DIR / "chat-art-platform-dataset" / "final_data"
WORKING_MODES: typing.List[WorkingMode] = [WorkingMode.TRAIN, WorkingMode.VAL]

DATASET_DIRS: typing.Dict[WorkingMode, Path] = {
    mode: FINAL_DATASET_DIR / mode
    for mode in WORKING_MODES
}
ANNOTATIONS_PATHS: typing.Dict[WorkingMode, Path] = {
    mode: FINAL_DATASET_DIR / f"{mode}.csv"
    for mode in WORKING_MODES
}
    
BATCH_SIZE: int = 64
N_WORKERS: int = 2
IMAGE_RESIZE_SIZE: int = 420

MODELS_DIR: Path = OUTPUT_DIR / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

MODEL_CLASS: typing.Type[nn.Module] = torch_models.efficientnet_b3
MODELS_WEIGHTS = torch_models.EfficientNet_B3_Weights.IMAGENET1K_V1

# Detect if we have a GPU available
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

# Создание модели pytorch

Выбрал модель классификации изображений https://pytorch.org/vision/stable/models.html#:~:text=EfficientNet_B3_Weights.IMAGENET1K_V1 как оптимальную по соотношению качество/скорость работы.

## Проверим модель

In [ ]:
def predict_by_model(img_path: Path, model: nn.Module, transforms=None) -> torch.Tensor:
    img = torch_io.read_image(str(img_path), mode=torch_io.image.ImageReadMode.RGB).to(DEVICE)
    if transforms:
        img = transforms(img)
    
    batch = img.unsqueeze(0)
    
    model.eval()
    prediction = model(batch).squeeze(0)
    
    return prediction

In [ ]:
img_path: Path = DATASET_DIRS[WorkingMode.TRAIN] / "19.png"

# Initialize model with the best available weights and the inference transforms
model: nn.Module = MODEL_CLASS(weights=MODELS_WEIGHTS).to(DEVICE)
transforms = MODELS_WEIGHTS.transforms()

prediction: torch.Tensor = predict_by_model(img_path=img_path, model=model, transforms=transforms)
probs: torch.Tensor = prediction.softmax(0)

# Use the model and print the predicted category
class_id: int = prediction.argmax().item()
score = probs[class_id].item()
category_name = MODELS_WEIGHTS.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")

## Load data

In [ ]:
class ImagePricingDataset(Dataset):

    def __init__(self, csv_path: Path, root_dir: Path, transform=None):
        """
        Args:
            csv_path: Path to the csv file with annotations.
            root_dir: directory with all the images.
            transform: Optional transform to be applied on a sample.
        """
        self.df: pd.DataFrame = pd.read_csv(csv_path)
        self.root_dir: Path = root_dir
        self.transform = transform

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        annotation: pd.Series = self.df.iloc[idx]
        
        img_path: Path = self.root_dir / annotation["image"]
        try:
            image = torch_io.read_image(str(img_path), mode=torch_io.image.ImageReadMode.RGB)
        except:
#             print(img_path)
            return self.__getitem__(idx=idx + 1)
        if self.transform:
            image = self.transform(image)
        
        price = annotation["price"]
        return {'image': image.float(), 'price': torch.tensor(price).float()}

### Посмотрим на примеры (проверим ImagePricingDataset)

In [ ]:
n_images: int = 5
common_transform = torch_transforms.Compose([
    torch_transforms.Resize(IMAGE_RESIZE_SIZE),
    MODELS_WEIGHTS.transforms(),
])
tmp_dataset: ImagePricingDataset = ImagePricingDataset(
    csv_path=ANNOTATIONS_PATHS[WorkingMode.TRAIN], 
    root_dir=DATASET_DIRS[WorkingMode.TRAIN], 
    transform=common_transform,
#     transform=None,
)
    
fig = plt.figure()
for i in range(len(tmp_dataset)):
    sample = tmp_dataset[i]

    print(i, sample['price'], sample['image'].shape, sample['price'].shape)

    ax = plt.subplot(1, n_images, i + 1)
    plt.tight_layout()
    ax.set_title(f"Sample #{i}, price={sample['price']}")
    ax.axis('off')
    plt.imshow(sample['image'].permute(1, 2, 0))

    if i == (n_images - 1):
        plt.show()
        break

DataLoader(ImagePricingDataset)

In [ ]:
tmp_dataset: ImagePricingDataset = ImagePricingDataset(
    csv_path=ANNOTATIONS_PATHS[WorkingMode.TRAIN], 
    root_dir=DATASET_DIRS[WorkingMode.TRAIN], 
    transform=torch_transforms.CenterCrop(200),
)
tmp_dataloader: DataLoader = DataLoader(tmp_dataset, batch_size=5, shuffle=False, num_workers=0)

for i, batch in enumerate(tmp_dataloader):
    print(i, batch["image"].size(), batch["price"].size(), batch["price"])
    if i == 2:
        break

## Аугментации

In [ ]:
AUGNENTATION_PROB: float = 0.5
GRAY_PROB: float = 0.2
BLUR_PROB: float = 1.0
NOISE_PROB: float = 0.2

BLUR_SIGMA: float = 2.0
BLUR_KERNEL_SIZE: typing.Tuple[int, int] = (21, 21)
NOISE_FACTOR: float = 0.1


def add_noise(inputs: torch.Tensor, noise_factor: float = 0.2) -> torch.Tensor:
    print("inputs:", inputs)
    max_noise: int = int(255 * noise_factor)
    noise = torch.randint_like(inputs.float(), high=max_noise)
    noisy = torch.clip(inputs + noise, 0, 255)
    print("noise:", noise)
    print("noisy:", noisy)
    return noisy

def noise_wrap(inputs: torch.Tensor) -> torch.Tensor:
    return add_noise(inputs=inputs, noise_factor=NOISE_FACTOR)


augmenters_list = [
    torch_transforms.RandomApply([torch_transforms.GaussianBlur(kernel_size=BLUR_KERNEL_SIZE, sigma=BLUR_SIGMA)], p=BLUR_PROB),
#     torch_transforms.RandomApply([torch_transforms.Lambda(noise_wrap)], p=NOISE_PROB),
    torch_transforms.RandomGrayscale(p=GRAY_PROB),
]

## DataLoadres

In [ ]:
print("Initializing Datasets and Dataloaders...")

common_transform = torch_transforms.Compose([
    torch_transforms.Resize(IMAGE_RESIZE_SIZE),
    MODELS_WEIGHTS.transforms(),
])
data_transforms = {
    WorkingMode.TRAIN: torch_transforms.Compose([
        torchvision.transforms.RandomApply(augmenters_list, p=AUGNENTATION_PROB), 
        common_transform,
    ]),
    WorkingMode.VAL: common_transform,
}

# Create training and validation datasets
image_datasets: typing.Dict[WorkingMode, ImagePricingDataset] = {
    mode: ImagePricingDataset(
        csv_path=ANNOTATIONS_PATHS[mode], 
        root_dir=DATASET_DIRS[mode], 
        transform=data_transforms[mode],
    )
    for mode in WORKING_MODES
}

# Create training and validation dataloaders
dataloaders: typing.Dict[WorkingMode, DataLoader] = {
    mode: DataLoader(image_datasets[mode], batch_size=BATCH_SIZE, shuffle=True, num_workers=N_WORKERS) 
    for mode in WORKING_MODES
}

# Создадим модель для дообучения

In [ ]:
def set_parameter_requires_grad(model: nn.Module, requires_grad: bool = False):
    for param in model.parameters():
        param.requires_grad = False

In [ ]:
def create_model(model_class: typing.Type[nn.Module], weights, freeze_body: bool = True) -> nn.Module:
    model: nn.Module = model_class(weights=weights)
    print(f"Orig classiffier layer (last layer): \n{model.classifier}\n")
    
    if freeze_body:
        set_parameter_requires_grad(model=model, requires_grad=False)

    n_in_features: int = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(n_in_features, 1)
    print(f"New classiffier layer (last layer): \n{model.classifier}")
    
    return model

In [ ]:
model: nn.Module = create_model(model_class=MODEL_CLASS, weights=MODELS_WEIGHTS, freeze_body=True)

## Model saving/loading

In [ ]:
def save_model_weights(model: nn.Module, model_name: str, model_dir: Path) -> Path:
    model_dir.mkdir(parents=True, exist_ok=True)
    weights_path: Path = model_dir / f"{model_name}_weights.pt"

    torch.save(model.state_dict(), weights_path)
    
    return weights_path


def load_from_weights(model: nn.Module, weights_path: Path) -> nn.Module:
    """Only weights."""
    model.load_state_dict(torch.load(weights_path))
    model.eval()
    return model

# def save_model(model: nn.Module, model_name: str, model_dir: Path) -> typing.Tuple[Path, Path]:
#     model_dir.mkdir(parents=True, exist_ok=True)
#     weights_path: Path = model_dir / f"{model_name}_weights.pt"
#     model_path: Path = model_dir / f"{model_name}_model.pt"

#     torch.save(model.state_dict(), weights_path)
#     torch.save(model, model_path)
    
#     return weights_path, model_path

# def load_model(model_path: Path) -> nn.Module:
#     """!!!Like pickle => use class name for loading => don't work, because we change last layer?."""
#     model: nn.Module = torch.load(final_model_path)
#     model.eval()
#     return model

In [ ]:
model_name: str = "dummy"
model_dir: Path = MODELS_DIR / model_name

dummy_model: nn.Module = create_model(model_class=MODEL_CLASS, weights=MODELS_WEIGHTS, freeze_body=True).to(DEVICE)
weights_path = save_model_weights(dummy_model, model_name=model_name, model_dir=model_dir)

# Load
new_model: nn.Module = create_model(model_class=MODEL_CLASS, weights=MODELS_WEIGHTS, freeze_body=True)
new_model = load_from_weights(model=new_model, weights_path=weights_path).to(DEVICE)
print("Loaded from weights")


# # Strict model
# other_model: nn.Module = load_model(model_path)
# print("Loaded model")

In [ ]:
img_path: Path = DATASET_DIRS[WorkingMode.TRAIN] / "19.png"
transforms = MODELS_WEIGHTS.transforms()

dummy_prediction: torch.Tensor = predict_by_model(img_path=img_path, model=dummy_model, transforms=transforms)
new_prediction: torch.Tensor = predict_by_model(img_path=img_path, model=new_model, transforms=transforms)
# other_prediction: torch.Tensor = predict_by_model(img_path=img_path, model=other_model, transforms=transforms)

print(dummy_prediction, new_prediction)

### Оптимизатор

In [ ]:
def create_optimizer(model, lr=0.001):
    print("Params to learn:")
    params_to_update = []
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)

    # Observe that all parameters are being optimized
    optimizer = optim.Adam(params_to_update, lr=lr)
    return optimizer

# Обучение модели

In [ ]:
def plot_hist(y_true: np.ndarray, y_pred: np.ndarray, title: str = None):
    preds = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})
    kwargs = dict(kind="hist", bins=100, alpha=0.5, figsize=(6, 3), title=title)
    preds.plot(**kwargs)
    plt.show()

In [ ]:
def rmsle(y_true: torch.Tensor, y_pred: torch.Tensor) -> torch.Tensor:
    return torch.mean((torch.log10(y_true) - torch.log10(y_pred))**2)

In [ ]:
def run_epoch(
    working_mode: WorkingMode, 
    model: nn.Module, 
    dataloader: DataLoader, 
    criterion, 
    optimizer, 
    num_epochs: int = 5,
) -> float:
    """Run one full epoch of train/val process."""
    start_date: datetime.datetime = datetime.datetime.now()
    print(f"{working_mode} start time: {start_date}")

    is_train: bool = (working_mode == WorkingMode.TRAIN)
    if is_train:
        model.train()  # Set model to training mode
    else:
        model.eval()   # Set model to evaluate mode

    sum_std_err_running_loss: float = 0.0
    sum_std_log_err_running_loss: float = 0.0
    dataset_size: int = len(dataloader.dataset)
    
    true_log_prices: list[np.ndarray] = []
    pred_log_prices: list[np.ndarray] = []

    # Iterate over data.
    for batch in tqdm(dataloader, total=dataset_size // dataloader.batch_size):
        inputs = batch["image"].to(DEVICE)
        prices = batch["price"].to(DEVICE)

        log_prices = torch.log10(prices)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        # track history if only in train
        with torch.set_grad_enabled(is_train):
            # Get model outputs and calculate loss

            log_pred_prices = model(inputs).reshape(-1)
            loss = criterion(log_pred_prices, log_prices)

            pred_prices = torch.pow(10, log_pred_prices).detach()

            # backward + optimize only if in training phase
            if is_train:
                loss.backward()
                optimizer.step()
            
            true_log_prices.append(log_prices.detach().to("cpu").numpy())
            pred_log_prices.append(log_pred_prices.detach().to("cpu").numpy())

        # statistics
        sum_std_err_running_loss += loss.to("cpu").item() * inputs.size(0)
        sum_std_log_err_running_loss += ((torch.log10(prices) - torch.log10(pred_prices))**2).sum().to("cpu").item()
#         break

    epoch_rmse: float = np.sqrt(sum_std_err_running_loss / dataset_size)
    epoch_rmsle: float = np.sqrt(sum_std_log_err_running_loss / dataset_size)
    
    plot_hist(
        y_true=np.concatenate(true_log_prices), 
        y_pred=np.concatenate(pred_log_prices),
        title=f"Log price; {working_mode}; RMSLE={epoch_rmsle:.4f}",
    )

    end_date: datetime.datetime = datetime.datetime.now()
    time_delta: datetime.timedelta = end_date - start_date
    print(f"Epoch complete in {time_delta}")
    print(f"{working_mode} RMSLE: {epoch_rmsle:.4f}, RMSE: {epoch_rmse:.4f}")
    
    return epoch_rmsle

In [ ]:
N_EPOCHES: int = 100
start_epoch: int = 48
LR: float = 0.001

model_name: str = "full_dataset_resize"
model_dir: Path = MODELS_DIR / model_name
interupted_models_dir: Path = MODELS_DIR / "interupted"

In [ ]:
model: nn.Module = create_model(model_class=MODEL_CLASS, weights=MODELS_WEIGHTS, freeze_body=True)

# load strict model
weights_path: Path = Path(
    '/kaggle/working/models/full_dataset/best_full_dataset_rmsle=0.4546_epoch=48_2023-04-14 16:10:34.150201_weights.pt'
)
model = load_from_weights(model=model, weights_path=weights_path)

model.to(DEVICE)

optimizer = create_optimizer(model=model, lr=LR)
criterion = nn.MSELoss()

In [ ]:
start_date: datetime.datetime = datetime.datetime.now()
print(f"Start time: {start_date}")

val_rmsle_history = []

best_model_wts = copy.deepcopy(model.state_dict())
best_rmsle = np.inf

for epoch in range(start_epoch, N_EPOCHES + 1):
    try:
        print("-" * 80)
        print(f"Epoch {epoch}/{N_EPOCHES}")
        print("-" * 80)

        # Each epoch has a training and validation phase
        for mode in WORKING_MODES:
            epoch_rmsle: float = run_epoch(
                working_mode=mode,
                model=model, 
                dataloader=dataloaders[mode], 
                criterion=criterion, 
                optimizer=optimizer,
            )

            # deep copy the model
            if mode == WorkingMode.VAL:
                val_rmsle_history.append(epoch_rmsle)

                if epoch_rmsle < best_rmsle:
                    best_rmsle = epoch_rmsle
                    best_model_wts = copy.deepcopy(model.state_dict())
                    weights_path = save_model_weights(
                        model=model, 
                        model_name=f"best_{model_name}_rmsle={best_rmsle:.4f}_epoch={epoch}_{datetime.datetime.now()}", 
                        model_dir=model_dir,
                    )

        end_date: datetime.datetime = datetime.datetime.now()
        time_delta: datetime.timedelta = end_date - start_date
        print(f"Epoch complete in {time_delta}")
        print(f"Best rmsle: {best_rmsle:4f}")
        print("\n")
        
    except KeyboardInterrupt:
        weights_path = save_model_weights(
            model=model, 
            model_name=f"INTERRUPTED_{model_name}_last_rmsle={epoch_rmsle:.4f}_epoch={epoch}_{datetime.datetime.now()}", 
            model_dir=interupted_models_dir,
        )
        print('Saved interrupted model')
        raise KeyboardInterrupt

# Выберем модель

In [ ]:
ls_command: str = f"ls {model_dir}"
print(ls_command)
os.system(ls_command)

In [ ]:
weights_path: Path = Path(
    '/kaggle/working/models/full_dataset/best_full_dataset_rmsle=0.4546_epoch=48_2023-04-14 16:10:34.150201_weights.pt'
)
new_model: nn.Module = create_model(model_class=MODEL_CLASS, weights=MODELS_WEIGHTS, freeze_body=True)
new_model = load_from_weights(model=new_model, weights_path=weights_path)
new_model.to(DEVICE)
pass

# Посмотрим прогнозы моделью

In [ ]:
img_path: Path = DATASET_DIRS[WorkingMode.TRAIN] / "35.png"
log_price: torch.Tensor = predict_by_model(img_path=img_path, model=model, transforms=transforms)
price: float = torch.pow(10, log_price).item()

print(f"{price:.2f}$")
Image.open(img_path)

# Запакуем в MLEM

## Предсказание по картинке

In [ ]:
import io
from PIL import Image

img_path: Path = DATASET_DIRS[WorkingMode.TRAIN] / "35.png"
img_bytes: bytes = img_path.read_bytes()


def mlem_predict(img_bytes: bytes):
    image = Image.open(io.BytesIO(img_bytes)).convert('RGB')
    transform = torch_transforms.Compose([torch_transforms.PILToTensor(), data_transforms[WorkingMode.VAL]])
    img: torch.Tensor = transform(image).to("cpu").float()
    batch: torch.Tensor = img.unsqueeze(0)
    
    new_model.to("cpu")
    new_model.eval()
    
    log_price: torch.Tensor = new_model(batch).squeeze(0)
    price: float = torch.pow(10, log_price).item()
    
    return {"price": price}

price: float = mlem_predict(img_bytes=img_bytes)["price"]
print(f"{price:.2f}$")
Image.open(img_path)

## Сохранение

In [ ]:
!pip install mlem --upgrade
!pip install mlem==0.4.6 --no-deps
!pip install iterative-telemetry==0.0.7 --ignore-requires-python --no-deps
!pip install pydantic==1.10.2 --no-deps

In [ ]:
import os
from mlem.api import save

mlem_path: Path = model_dir / f"mlem_{model_name}"
save(
    mlem_predict, 
    mlem_path, 
    sample_data=img_bytes,
);

In [ ]:
zip_command: str = f"zip -r {mlem_path.name}.zip {mlem_path}.mlem {mlem_path}"
print(zip_command)
os.system(zip_command)

In [ ]:
mlem_path

## Скачаем zip с mlem моделью

In [ ]:
from IPython.display import FileLink
os.chdir(OUTPUT_DIR)
FileLink(f"{mlem_path.name}.zip")